In [ ]:
import numpy as np
import torch
import torch.nn as nn

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Make Orig Model

### Download MiDaS

In [ ]:
!git clone https://github.com/intel-isl/MiDaS.git

Cloning into 'MiDaS'...
remote: Enumerating objects: 250, done.
remote: Counting objects: 100% (250/250), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 394 (delta 58), reused 201 (delta 37), pack-reused 144
Receiving objects: 100% (394/394), 231.02 KiB | 13.59 MiB/s, done.
Resolving deltas: 100% (139/139), done.


In [ ]:
%cd MiDaS

/content/MiDaS


### Create the complete model

In [ ]:
!cp '../drive/My Drive/eva_stored_from_colab/eva5/s15_capstone/modified_from_orig/run.py' .

In [ ]:
!cp '../drive/My Drive/eva_stored_from_colab/eva5/s15_capstone/model-f6b98070.pt' .

In [ ]:
import run
orig_model = run.run("input", "output", "model-f6b98070.pt")

initialize
device: cpu
Loading weights:  model-f6b98070.pt


Downloading: "https://github.com/facebookresearch/WSL-Images/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://download.pytorch.org/models/ig_resnext101_32x8-c38310e5.pth" to /root/.cache/torch/hub/checkpoints/ig_resnext101_32x8-c38310e5.pth


## New Model

In [ ]:

def _make_scratch(in_shape, out_shape, groups=1, expand=False):
    scratch = nn.Module()

    out_shape1 = out_shape
    out_shape2 = out_shape
    out_shape3 = out_shape
    out_shape4 = out_shape
    if expand==True:
        out_shape1 = out_shape
        out_shape2 = out_shape*2
        out_shape3 = out_shape*4
        out_shape4 = out_shape*8

    scratch.layer1_rn = nn.Conv2d(
        in_shape[0], out_shape1, kernel_size=3, stride=1, padding=1, bias=False, groups=groups
    )
    scratch.layer2_rn = nn.Conv2d(
        in_shape[1], out_shape2, kernel_size=3, stride=1, padding=1, bias=False, groups=groups
    )
    scratch.layer3_rn = nn.Conv2d(
        in_shape[2], out_shape3, kernel_size=3, stride=1, padding=1, bias=False, groups=groups
    )
    scratch.layer4_rn = nn.Conv2d(
        in_shape[3], out_shape4, kernel_size=3, stride=1, padding=1, bias=False, groups=groups
    )

    return scratch




class Interpolate(nn.Module):
    """Interpolation module.
    """

    def __init__(self, scale_factor, mode):
        """Init.

        Args:
            scale_factor (float): scaling
            mode (str): interpolation mode
        """
        super(Interpolate, self).__init__()

        self.interp = nn.functional.interpolate
        self.scale_factor = scale_factor
        self.mode = mode

    def forward(self, x):
        """Forward pass.

        Args:
            x (tensor): input

        Returns:
            tensor: interpolated data
        """

        x = self.interp(
            x, scale_factor=self.scale_factor, mode=self.mode, align_corners=False
        )

        return x



class ResidualConvUnit(nn.Module):
    """Residual convolution module.
    """

    def __init__(self, features):
        """Init.

        Args:
            features (int): number of features
        """
        super().__init__()

        self.conv1 = nn.Conv2d(
            features, features, kernel_size=3, stride=1, padding=1, bias=True
        )

        self.conv2 = nn.Conv2d(
            features, features, kernel_size=3, stride=1, padding=1, bias=True
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        """Forward pass.

        Args:
            x (tensor): input

        Returns:
            tensor: output
        """
        out = self.relu(x)
        out = self.conv1(out)
        out = self.relu(out)
        out = self.conv2(out)

        return out + x



class FeatureFusionBlock(nn.Module):
    """Feature fusion block.
    """

    def __init__(self, features):
        """Init.
        Args:
            features (int): number of features
        """
        super(FeatureFusionBlock, self).__init__()

        self.resConfUnit1 = ResidualConvUnit(features)
        self.resConfUnit2 = ResidualConvUnit(features)

    def forward(self, *xs):
        """Forward pass.
        Returns:
            tensor: output
        """
        output = xs[0]

        if len(xs) == 2:
            output += self.resConfUnit1(xs[1])

        output = self.resConfUnit2(output)

        output = nn.functional.interpolate(
            output, scale_factor=2, mode="bilinear", align_corners=True
        )

        return output




class ResidualConvUnit_custom(nn.Module):
    """Residual convolution module.
    """

    def __init__(self, features, activation, bn):
        """Init.
        Args:
            features (int): number of features
        """
        super().__init__()

        self.bn = bn

        self.groups=1

        self.conv1 = nn.Conv2d(
            features, features, kernel_size=3, stride=1, padding=1, bias=True, groups=self.groups
        )
        
        self.conv2 = nn.Conv2d(
            features, features, kernel_size=3, stride=1, padding=1, bias=True, groups=self.groups
        )

        if self.bn==True:
            self.bn1 = nn.BatchNorm2d(features)
            self.bn2 = nn.BatchNorm2d(features)

        self.activation = activation

        self.skip_add = nn.quantized.FloatFunctional()

    def forward(self, x):
        """Forward pass.
        Args:
            x (tensor): input
        Returns:
            tensor: output
        """
        
        out = self.activation(x)
        out = self.conv1(out)
        if self.bn==True:
            out = self.bn1(out)
       
        out = self.activation(out)
        out = self.conv2(out)
        if self.bn==True:
            out = self.bn2(out)

        if self.groups > 1:
            out = self.conv_merge(out)

        return self.skip_add.add(out, x)



class MidasScratch(nn.Module):

    def __init__(self, path=None, features=256, non_negative=True):
        """Init.
        Args:
            path (str, optional): Path to saved model. Defaults to None.
            features (int, optional): Number of features. Defaults to 256.
            backbone (str, optional): Backbone network for encoder. Defaults to resnet50
        """
        print("Loading weights: ", path)

        super(MidasScratch, self).__init__()

        use_pretrained = False if path is None else True

        self.scratch = _make_scratch(in_shape=[256, 512, 1024, 2048], 
                                     out_shape=features, groups=1, expand=False)


        self.scratch.refinenet4 = FeatureFusionBlock(features)
        self.scratch.refinenet3 = FeatureFusionBlock(features)
        self.scratch.refinenet2 = FeatureFusionBlock(features)
        self.scratch.refinenet1 = FeatureFusionBlock(features)

        self.scratch.output_conv = nn.Sequential(
            nn.Conv2d(features, 128, kernel_size=3, stride=1, padding=1),
            Interpolate(scale_factor=2, mode="bilinear"),
            nn.Conv2d(128, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
            nn.Conv2d(32, 1, kernel_size=1, stride=1, padding=0),
            nn.ReLU(True) if non_negative else nn.Identity(),
        )

        if path:
            self.load(path)


    def forward(self, layer_1, layer_2, layer_3, layer_4):
        """Forward pass.
        Args:
            x (tensor): input data (image)
        Returns:
            tensor: depth
        """

        layer_1_rn = self.scratch.layer1_rn(layer_1)
        layer_2_rn = self.scratch.layer2_rn(layer_2)
        layer_3_rn = self.scratch.layer3_rn(layer_3)
        layer_4_rn = self.scratch.layer4_rn(layer_4)

        path_4 = self.scratch.refinenet4(layer_4_rn)
        path_3 = self.scratch.refinenet3(path_4, layer_3_rn)
        path_2 = self.scratch.refinenet2(path_3, layer_2_rn)
        path_1 = self.scratch.refinenet1(path_2, layer_1_rn)

        out = self.scratch.output_conv(path_1)

        return torch.squeeze(out, dim=1)



In [ ]:
new_model = MidasScratch()

Loading weights:  None


## Extract weights from old model and assign to new model

In [ ]:
def extract_relevant_midas_weights(orig_model, new_model):

    layer_names = [
    "scratch.layer1_rn",
    "scratch.layer2_rn",
    "scratch.layer3_rn",
    "scratch.layer4_rn",
    "scratch.refinenet4.resConfUnit1.conv1",
    "scratch.refinenet4.resConfUnit1.conv2",
    "scratch.refinenet4.resConfUnit2.conv1",
    "scratch.refinenet4.resConfUnit2.conv2",
    "scratch.refinenet3.resConfUnit1.conv1",
    "scratch.refinenet3.resConfUnit1.conv2",
    "scratch.refinenet3.resConfUnit2.conv1",
    "scratch.refinenet3.resConfUnit2.conv2",
    "scratch.refinenet2.resConfUnit1.conv1",
    "scratch.refinenet2.resConfUnit1.conv2",
    "scratch.refinenet2.resConfUnit2.conv1",
    "scratch.refinenet2.resConfUnit2.conv2",
    "scratch.refinenet1.resConfUnit1.conv1",
    "scratch.refinenet1.resConfUnit1.conv2",
    "scratch.refinenet1.resConfUnit2.conv1",
    "scratch.refinenet1.resConfUnit2.conv2",
    "scratch.output_conv[0]",
    "scratch.output_conv[2]",
    "scratch.output_conv[4]"]

    for lyr in layer_names:
        exec("new_model." + lyr + ".weight.data = orig_model." + lyr + ".weight.data")
    
    return new_model


In [ ]:
new_model = extract_relevant_midas_weights(orig_model, new_model)

In [ ]:
for k, l in new_model.named_parameters():
    print()
    print("************************************")
    print(k, "  --  ", l.shape)
    print("************************************")

In [ ]:
torch.save(new_model.state_dict(), '../drive/My Drive/eva_stored_from_colab/eva5/s15_capstone/midas_scratch_weights.pt')

In [ ]:
testmodel = MidasScratch()
testmodel.load_state_dict(torch.load('../drive/My Drive/eva_stored_from_colab/eva5/s15_capstone/midas_scratch_weights.pt'))

Loading weights:  None


<All keys matched successfully>

In [ ]:
for k, l in testmodel.named_parameters():
    print()
    print("************************************")
    print(k, "  --  ", l.shape)
    print("************************************")


************************************
scratch.layer1_rn.weight   --   torch.Size([256, 256, 3, 3])
************************************

************************************
scratch.layer2_rn.weight   --   torch.Size([256, 512, 3, 3])
************************************

************************************
scratch.layer3_rn.weight   --   torch.Size([256, 1024, 3, 3])
************************************

************************************
scratch.layer4_rn.weight   --   torch.Size([256, 2048, 3, 3])
************************************

************************************
scratch.refinenet4.resConfUnit1.conv1.weight   --   torch.Size([256, 256, 3, 3])
************************************

************************************
scratch.refinenet4.resConfUnit1.conv1.bias   --   torch.Size([256])
************************************

************************************
scratch.refinenet4.resConfUnit1.conv2.weight   --   torch.Size([256, 256, 3, 3])
************************************

## Storing parameters as list of named tuple

In [ ]:
def extract_relevant_midas_weights(orig_model, new_model):

    layer_names = [
    "scratch.layer1_rn",
    "scratch.layer2_rn",
    "scratch.layer3_rn",
    "scratch.layer4_rn",
    "scratch.refinenet4.resConfUnit1.conv1",
    "scratch.refinenet4.resConfUnit1.conv2",
    "scratch.refinenet4.resConfUnit2.conv1",
    "scratch.refinenet4.resConfUnit2.conv2",
    "scratch.refinenet3.resConfUnit1.conv1",
    "scratch.refinenet3.resConfUnit1.conv2",
    "scratch.refinenet3.resConfUnit2.conv1",
    "scratch.refinenet3.resConfUnit2.conv2",
    "scratch.refinenet2.resConfUnit1.conv1",
    "scratch.refinenet2.resConfUnit1.conv2",
    "scratch.refinenet2.resConfUnit2.conv1",
    "scratch.refinenet2.resConfUnit2.conv2",
    "scratch.refinenet1.resConfUnit1.conv1",
    "scratch.refinenet1.resConfUnit1.conv2",
    "scratch.refinenet1.resConfUnit2.conv1",
    "scratch.refinenet1.resConfUnit2.conv2",
    "scratch.output_conv[0]",
    "scratch.output_conv[2]",
    "scratch.output_conv[4]"]

    list_of_weights = []
    for lyr in layer_names:
        tup = ( lyr, eval("orig_model." + lyr + ".weight.data.numpy()") )
        list_of_weights.append(tup)
    
    return list_of_weights


In [ ]:
lots = extract_relevant_midas_weights(orig_model, new_model=None)

In [ ]:
import pickle

with open("../drive/My Drive/eva_stored_from_colab/eva5/s15_capstone/midas_scratch_list_of_tuples.pkl", "wb") as pklfl:
    pickle.dump(lots, pklfl)

In [ ]:
with open("../drive/My Drive/eva_stored_from_colab/eva5/s15_capstone/midas_scratch_list_of_tuples.pkl", "rb") as pklfl:
    testmdl = pickle.load(pklfl)

In [ ]:
print(type(testmdl))
for k in testmdl:
    print(type(k[0]), type(k[1]))
    print(k)